In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from scipy.stats import boxcox
%matplotlib inline

In [2]:
sales_of_company_x = pd.read_csv("./data/monthly-sales-of-company-x-jan-6.csv")
robberies_in_boston = pd.read_csv("./data/monthly-boston-armed-robberies-j.csv")
mean_monthly_temp = pd.read_csv("./data/mean-monthly-air-temperature-deg.csv")
dowjones_closing = pd.read_csv("./data/weekly-closings-of-the-dowjones-.csv")
female_births = pd.read_csv("./data/daily-total-female-births-in-cal.csv")

In [3]:
sales_of_company_x

,Month,Count
0,1965-01,154
1,1965-02,96
2,1965-03,73
3,1965-04,49
4,1965-05,36
...,...,...
72,1971-01,628
73,1971-02,308
74,1971-03,324
75,1971-04,248
